## Google Drive Authentication

In [15]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive']


creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

try:
    service = build('drive', 'v3', credentials=creds)
    service_sheets = build('sheets', 'v4', credentials=creds)
    
    print("** Authentication Complete! **")

except HttpError as error:
    # TODO(developer) - Handle errors from drive API.
    print(f'An error occurred: {error}')


** Authentication Complete! **


## Google Drive API helper functions

In [18]:
import io
from googleapiclient.http import MediaIoBaseDownload
from os import listdir

def getFolderId(service, folderName: str):
    query = "name contains '%s' and mimeType = '%s'" % (folderName, 'application/vnd.google-apps.folder')

    fid = None

    if folderName.startswith('+'):
        return (folderName[1:])

    result = service.files().list(q=query,
                                  pageSize=10, pageToken='',
                                  fields="nextPageToken,files(parents,id,name,mimeType)").execute()
  
    if len(result['files']) == 0:
        print("Folder NOT found")
    else:
        folder = result.get('files')[0]
        fid = folder['id']

    return (fid)

    
def downloadFolder(service, fileId, destinationFolder):
    if not os.path.isdir(destinationFolder):
        os.mkdir(path=destinationFolder)

    results = service.files().list(
        pageSize=300,
        q="parents in '{0}'".format(fileId),
        fields="files(id, name, mimeType)"
        ).execute()

    items = results.get('files', [])

    for item in items:
        itemName = item['name']
        itemId = item['id']
        itemType = item['mimeType']
        filePath = destinationFolder + "/" + itemName

        if itemType == 'application/vnd.google-apps.folder':
            downloadFolder(service, itemId, filePath) # Recursive call
            print("Downloaded folder: {0}".format(itemName))
        elif not itemType.startswith('application/'):
            downloadFile(service, itemId, filePath)
        else:
            print("Unsupported file: {0}".format(itemName))


def downloadFile(service, fileId, filePath):
    # Note: The parent folders in filePath must exist
    request = service.files().get_media(fileId=fileId)
    fh = io.FileIO(filePath, mode='wb')
    
    try:
        downloader = MediaIoBaseDownload(fh, request, chunksize=1024*1024)

        done = False
        while done is False:
            status, done = downloader.next_chunk(num_retries = 2)
    finally:
        fh.close()
        
def deleteFilesInFolder(folder_id):
    results = service.files().list(
        pageSize=300,
        q="parents in '{0}'".format(folder_id),
        fields="files(id, name, mimeType)"
        ).execute()

    items = results.get('files', [])
    for item in items:
        itemId = item['id']
        service.files().delete(fileId=itemId).execute()
        
def uploadFolder(service, folder_id, src_folder):        
    for file in listdir(src_folder):
        print('Uploading: ' + file)
        file_metadata = {'name': file, 'parents': [folder_id]}
        media = MediaFileUpload(os.path.join(src_folder, file), mimetype='image/png')
        file = service.files().create(body=file_metadata,
                                    media_body=media,
                                    fields='id').execute()

## Download Trait Files!

In [19]:
import shutil

traits_base_filepath = 'Traits'

# Delete previously downloaded trait files
if os.path.isdir(traits_base_filepath):
    shutil.rmtree(traits_base_filepath)

traits_folder_id = getFolderId(service, traits_base_filepath)

downloadFolder(service, traits_folder_id, traits_base_filepath)

print("\n** Download Complete! **")

Downloaded folder: earrings
Downloaded folder: ears
Downloaded folder: quarter zip
Downloaded folder: upper + lower
Downloaded folder: short pants
Downloaded folder: hats + face
Downloaded folder: headphones
Downloaded folder: spilled carton
Downloaded folder: bucket hats
Downloaded folder: Flag Patches
Downloaded folder: Patches
Downloaded folder: beanies
Downloaded folder: Patches
Downloaded folder: baseball cap
Downloaded folder: hats
Downloaded folder: dots
Downloaded folder: healthbar
Downloaded folder: external
Downloaded folder: syringe
Downloaded folder: sticky notes
Downloaded folder: glasses
Downloaded folder: fb
Downloaded folder: surgical masks
Downloaded folder: face
Downloaded folder: Meta Tees
Downloaded folder: short sleeves
Downloaded folder: fat pants
Downloaded folder: NB Sneakers
Downloaded folder: AirForce1
Downloaded folder: shoes
Downloaded folder: Sweatpants LP
Downloaded folder: Sweatpants Plain
Downloaded folder: slim pants
Downloaded folder: undergarment uppe

## Some more helper functions

In [30]:
from collections import deque
import random

def pick_trait_images(folder):
    
    files = os.listdir(folder)
    files = filter(lambda file: not file.startswith('.'), files)
    
    png_files = []
    folder_files = []
    for file in files:
        if file.lower().endswith('.png'):
            png_files.append(file)
        else:
            folder_files.append(file)
    
    chosen_image = rarity_chooser(png_files)
    chosen_folder = rarity_chooser(folder_files)
    
    # If only PNGs in folder
    if len(folder_files) == 0:
        return [os.path.join(folder, chosen_image)]
    
    # If only folders in folder
    elif len(png_files) == 0:
        return pick_trait_images(os.path.join(folder, chosen_folder))
    
    # Both PNGs and folders in folder
    else:        
        return [os.path.join(folder, chosen_image)] + pick_trait_images(os.path.join(folder, chosen_folder))

# Randomly choosing for now
def rarity_chooser(elems):
    if elems:
        return random.choice(elems)
    return None

## Generate Babies!

In [31]:
sheet = service_sheets.spreadsheets()

LAYERING_ORDER_SPREADSHEET_ID = '1aHC5g3mPSJGFAPF7UiQXDnV9BBUJCwnx8QHkMMip6uI'
RANGE = 'A1:A'


result = sheet.values().get(spreadsheetId=LAYERING_ORDER_SPREADSHEET_ID, range=RANGE).execute()
values = result.get('values', [])

ordered_traits = [item for sublist in values for item in sublist]

print("Traits Ordering: \n")
for trait in ordered_traits:
    print(trait + '\n')

images_count = 10

babies_base_filepath = 'Babies'

Traits Ordering: 

body

upper + lower

undergarment lower

undergarment upper

short pants

slim pants

shoes

fat pants

short sleeves

dress shirt

quarter zip

hoodies

face

hats

hats + face

ears

external



In [36]:
from PIL import Image

def generate_baby(ordered_traits, traits_base_filepath, babies_base_filepath):
    
    # BFS for trait images selection
    picked_trait_images = [] 
    base_traits = ordered_traits 

    while base_traits:
        curr = base_traits.pop(0)
        curr_trait_dir = os.path.join(traits_base_filepath, curr)
        picked_trait_images.extend(pick_trait_images(curr_trait_dir))

    # Layer the images 
    x, y = Image.open(picked_trait_images[0]).size
    final_baby_image = Image.new('RGB', (x, y), (228, 150, 150))

    for trait_image in picked_trait_images:
        chosen_image = Image.open(trait_image)
        final_baby_image.paste(chosen_image, (0, 0), chosen_image)

    return final_baby_image


## Generate babies!

# Delete previously created baby files
if os.path.isdir(babies_base_filepath):
    shutil.rmtree(babies_base_filepath)

os.mkdir(babies_base_filepath)

for i in range(images_count):
    final_baby_image = generate_baby(ordered_traits.copy(), traits_base_filepath, babies_base_filepath)
    
    # Write the image to file
    final_baby_image_file = '{:d}_lonely_baby.PNG'.format(i)
    final_baby_image.save(os.path.join(babies_base_filepath, final_baby_image_file))
    
    print("Completed " + final_baby_image_file)

print("\n**Image generation complete! **")

Completed 0_lonely_baby.PNG
Completed 1_lonely_baby.PNG
Completed 2_lonely_baby.PNG
Completed 3_lonely_baby.PNG
Completed 4_lonely_baby.PNG
Completed 5_lonely_baby.PNG
Completed 6_lonely_baby.PNG
Completed 7_lonely_baby.PNG
Completed 8_lonely_baby.PNG
Completed 9_lonely_baby.PNG

**Image generation complete! **


## Upload Babies to Google Drive

In [23]:
babies_folder_id = getFolderId(service, babies_base_filepath)
deleteFilesInFolder(babies_folder_id)
uploadFolder(service, babies_folder_id, os.path.join(babies_base_filepath))

25_lonely_baby.PNG
18_lonely_baby.PNG
8_lonely_baby.PNG
46_lonely_baby.PNG
33_lonely_baby.PNG
49_lonely_baby.PNG
7_lonely_baby.PNG
17_lonely_baby.PNG
10_lonely_baby.PNG
0_lonely_baby.PNG
34_lonely_baby.PNG
41_lonely_baby.PNG
22_lonely_baby.PNG
11_lonely_baby.PNG
1_lonely_baby.PNG
35_lonely_baby.PNG
40_lonely_baby.PNG
23_lonely_baby.PNG
24_lonely_baby.PNG
19_lonely_baby.PNG
9_lonely_baby.PNG
47_lonely_baby.PNG
48_lonely_baby.PNG
32_lonely_baby.PNG
6_lonely_baby.PNG
16_lonely_baby.PNG
37_lonely_baby.PNG
3_lonely_baby.PNG
13_lonely_baby.PNG
21_lonely_baby.PNG
38_lonely_baby.PNG
42_lonely_baby.PNG
45_lonely_baby.PNG
26_lonely_baby.PNG
29_lonely_baby.PNG
14_lonely_baby.PNG
4_lonely_baby.PNG
30_lonely_baby.PNG
44_lonely_baby.PNG
27_lonely_baby.PNG
28_lonely_baby.PNG
15_lonely_baby.PNG
5_lonely_baby.PNG
31_lonely_baby.PNG
36_lonely_baby.PNG
2_lonely_baby.PNG
12_lonely_baby.PNG
20_lonely_baby.PNG
43_lonely_baby.PNG
39_lonely_baby.PNG
